# Color maps to CSS gradients

Colormaps are functions that map from `[0, 1]` to colors. I love colormaps! Maybe a little too much? There is a huge amount of [very serious color science](https://colorcet.com) behind colormaps, and also a great Python package called [colorcet](https://github.com/holoviz/colorcet) which implements a lot of them.

Would you ever need those as a CSS gradient? Probably not. Did I have fun writing a script to convert every 120 of them to a CSS gradient? Absolutely! Enjoy 😁

> Note that these sample the actual colormap at N=101 points, and then your browser does [linear interpolation](https://developer.mozilla.org/en-US/docs/Web/CSS/color-interpolation-method) to fill in the gaps. The actual colormaps are [255-entry tables](https://matplotlib.org/stable/api/_as_gen/matplotlib.colors.LinearSegmentedColormap.html). So not only is this horribly inefficient by using very verbose CSS syntax, it's also destroying a lot of the original colormap's information. This is just for fun, use the original data for anything serious.

> Also, I excluded the categorical colormaps because they don't really make sense as a gradient.

In [28]:
a = cc.cm.CET_R4
b = mpl.colormaps["cet_CET_R4"]

print(set([type(c) for c in cc.cm.values()]))
print(set([type(c) for c in mpl.colormaps.values()]))

{<class 'matplotlib.colors.LinearSegmentedColormap'>, <class 'matplotlib.colors.ListedColormap'>}
{<class 'matplotlib.colors.LinearSegmentedColormap'>, <class 'matplotlib.colors.ListedColormap'>}


In [1]:
import matplotlib as mpl
import colorcet as cc

from collections import defaultdict

print("matplotlib version", mpl.__version__)
print("colorcet version", cc.__version__)

all_cmaps = {}
other_names = defaultdict(set)

def allowed(names):
    for n in names:
        if n.endswith("_r"):
            return False
        if "glasbey" in n:
            return False
    return True

for name, cmap in cc.cm.items():
    if allowed([name, cmap.name]):
        all_cmaps[cmap.name] = cmap
        other_names[cmap.name].update([name, cmap.name])

for name, cmap in mpl.colormaps.items():
    if allowed([name, cmap.name]):
        all_cmaps[cmap.name] = cmap
        other_names[cmap.name].update([name, cmap.name])

print(len(all_cmaps))
print(len(other_names))

matplotlib version 3.9.4
colorcet version 3.1.0
397
397


In [5]:
import matplotlib as mpl
import colorcet as cc
import numpy as np
from IPython.display import display, HTML


def background_css(cmap, N):
    N = 101
    
    def pl(v):
        return map(int, np.round(np.array(cmap(v))*255))
    
    space = np.linspace(0, 1, N)
    domain = np.linspace(0, 1, N)
    
    s = "background: linear-gradient(90deg, "
    for i, j in zip(space, domain):
        r, g, b, a = pl(i)
        hex_str = "#{:02x}{:02x}{:02x}".format(r,g,b)
        s += f"{hex_str} {round(100*j)}%, "

    s = s[:-2]
    s += ");"

    return s

html_template = '<div class="gradient" style="{}"></div>'

total_md = """
<style>
div.gradient {
    width: 650px;
    height: 60px;
    border-radius: 3px;
    margin-bottom: 10px;
    box-shadow: 5px 5px 5px rgb(0, 0, 0, 0.5);
    margin: 0 auto;
}
pre {
max-width: 600px;
}
</style>

"""

for name, cmap in sorted(all_cmaps.items()):
    names = list(sorted(other_names[name]))

    css = background_css(cmap, N=6)
    html = html_template.format(css)
    total_md += f'\n## {" / ".join(names)}\n'
    total_md += f'\n{html}\n'
    #total_md += f'\n```\n{css}\n```\n'

with open("css_gradients.md", "w") as f:
    f.write(total_md)


